<a href="https://colab.research.google.com/github/monicasofiarestrepo/SimulacionDeSIstemas/blob/main/simulacion_eventos_discretos_simpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ej.4.3 Banks. A la  caja de una tienda llegan clientes con un tiempo entre llegadas distribuido como exponencial con media de 4.5 minutos y los tiempos de servicio de la única caja se distribuyen de manera aproximadamente normal con media de 3.2 minutos y desviación estándar de 0.6 minutos. La aproximación de los tiempos de servicio es siempre positiva.

Cuando el cajero está ocupado, los clientes se hacen en cola y se supone que ningún cliente se va. Simular la atención de 1000 clientes.

In [ ]:
!pip install simpy

In [ ]:
import simpy
import random

RANDOM_SEED = 42
NEW_CUSTOMERS = 10  # número total de clientes
INTERVAL_CUSTOMERS = 4.5  # Generar nuevos clientes cada x minutos

# Generación de llegadas distribuidas exponencialmente
def llegadas(env,num,tasa,caja):
  for i in range(num):
    c=cliente(env, 'Cliente%02d'% i, caja, mus=3.2,stds=0.6 )
    env.process(c)
    t = random.expovariate(1.0 / tasa)
    yield env.timeout(t)
# Cada cliente requiere la caja y tiene un tiempo de servicio aprox. N(mu,stds)
def cliente(env, name, caja, mus,stds):
    #llega un cliente, registrar momento
    arrive = env.now
    print('%7.4f %s: Llegó\n' % (arrive, name))
    with caja.request() as req:
        yield req
        wait = env.now - arrive
        print('%7.4f %s: Esperó %6.3f\n' % (env.now, name, wait))
        tib=random.normalvariate(mus,stds)
        yield env.timeout(tib)
        print('%7.4f %s: Acabó\n' % (env.now, name))


# Simulación, configuración
print('Caja\n')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Ejecución de los procesos y la simulación
caja = simpy.Resource(env, capacity=1)
env.process(llegadas(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, caja))
env.run()

Caja

 0.0000 Cliente00: Llegó

 0.0000 Cliente00: Esperó  0.000

 2.5256 Cliente00: Acabó

 4.5903 Cliente01: Llegó

 4.5903 Cliente01: Esperó  0.000

 5.7269 Cliente02: Llegó

 8.5431 Cliente01: Acabó

 8.5431 Cliente02: Esperó  2.816

11.0077 Cliente02: Acabó

15.7497 Cliente03: Llegó

15.7497 Cliente03: Esperó  0.000

15.8858 Cliente04: Llegó

16.0069 Cliente05: Llegó

17.0045 Cliente06: Llegó

18.3642 Cliente03: Acabó

18.3642 Cliente04: Esperó  2.478

21.6236 Cliente04: Acabó

21.6236 Cliente05: Esperó  5.617

21.7272 Cliente07: Llegó

21.7565 Cliente08: Llegó

25.3057 Cliente05: Acabó

25.3057 Cliente06: Esperó  8.301

28.8148 Cliente06: Acabó

28.8148 Cliente07: Esperó  7.088

29.1319 Cliente09: Llegó

31.8294 Cliente07: Acabó

31.8294 Cliente08: Esperó 10.073

35.9321 Cliente08: Acabó

35.9321 Cliente09: Esperó  6.800

40.3018 Cliente09: Acabó



1. Identificar los eventos en el código anterior

2. Identificar los datos de entrada del código anterior

3. Con base en la guía suministrada, modificar el código para que un cliente que espere más de 10 minutos en cola abandone

4. Modificar el código para calcular número de abandonos, tiempo promedio en el sistema, tiempo promedio en cola contando únicamente los que esperaron y contando todos los clientes.

5. Con base en el ejemplo de la presentación de introducción a GPSS de la sección anterior (https://docs.google.com/presentation/d/1LNSwwpVfK_O6Rq-wtjtD7-cNqe7dA00DzfpfjdV5kqk/edit?usp=sharing), reproducir el modelo en GPSS y analizar los resultados.

#Identificar los eventos en el código anterior

Los eventos del codigo anterior (el original de la práctica sin modificaciones)son:
Caja

 0.0000 Cliente00: Llegó

 0.0000 Cliente00: Esperó  0.000

 2.5256 Cliente00: Acabó

 4.5903 Cliente01: Llegó

 4.5903 Cliente01: Esperó  0.000

 5.7269 Cliente02: Llegó

 8.5431 Cliente01: Acabó

 8.5431 Cliente02: Esperó  2.816

11.0077 Cliente02: Acabó

15.7497 Cliente03: Llegó

15.7497 Cliente03: Esperó  0.000

15.8858 Cliente04: Llegó

16.0069 Cliente05: Llegó

17.0045 Cliente06: Llegó

18.3642 Cliente03: Acabó

18.3642 Cliente04: Esperó  2.478

21.6236 Cliente04: Acabó

21.6236 Cliente05: Esperó  5.617

21.7272 Cliente07: Llegó

21.7565 Cliente08: Llegó

25.3057 Cliente05: Acabó

25.3057 Cliente06: Esperó  8.301

28.8148 Cliente06: Acabó

28.8148 Cliente07: Esperó  7.088

29.1319 Cliente09: Llegó

31.8294 Cliente07: Acabó

31.8294 Cliente08: Esperó 10.073

35.9321 Cliente08: Acabó

35.9321 Cliente09: Esperó  6.800

40.3018 Cliente09: Acabó

 En SimPy un evento es una representación de un suceso específico en un sistema de simulación, y se utiliza para controlar y modelar el flujo del tiempo y el comportamiento del sistema. Los eventos puestos arriba traen su estampa de tiempo y descripción.

#Identificar los datos de entrada del código anterior

1. Número total de clientes (NEW_CUSTOMERS)
2. El intervalo entre llegadas de clientes (INTERVAL_CUSTOMERS)
3. Los parámetros de distribución para el tiempo de servicio de los clientes (mus y stds).

In [ ]:
#@title modificar el código para que un cliente que espere más de 10 minutos en cola abandone

import simpy
import random

RANDOM_SEED = 42
NEW_CUSTOMERS = 10  # número total de clientes
INTERVAL_CUSTOMERS = 4.5  # Generar nuevos clientes cada x minutos
MAX_WAIT_TIME = 10.0  # Máximo tiempo de espera en minutos, son 10 por requerimientos del ejercicio

# Generación de llegadas distribuidas exponencialmente
def llegadas(env, num, tasa, caja):
    for i in range(num):
        c = cliente(env, 'Cliente%02d' % i, caja, mus=3.2, stds=0.6)
        env.process(c)
        t = random.expovariate(1.0 / tasa)
        yield env.timeout(t)

# Cada cliente requiere la caja y tiene un tiempo de servicio aprox. N(mu, stds)
def cliente(env, name, caja, mus, stds):
    # Llega un cliente, registrar momento
    arrive = env.now
    print('%7.4f %s: Llegó' % (arrive, name))

    with caja.request() as req:
        yield req
        wait = env.now - arrive
        print('%7.4f %s: Esperó %6.3f' % (env.now, name, wait))

        # si el cliente ha esperado más de 10 minutos abandona la cola y no completa su tiempo de servicio.
        if wait > MAX_WAIT_TIME:
            print('%7.4f %s: Abandonó después de esperar %6.3f' % (env.now, name, wait))
            return

        tib = random.normalvariate(mus, stds)
        yield env.timeout(tib)
        print('%7.4f %s: Acabó' % (env.now, name))

# Simulación, configuración
print('Caja\n')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Ejecución de los procesos y la simulación
caja = simpy.Resource(env, capacity=1)
env.process(llegadas(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, caja))
env.run()

Caja

 0.0000 Cliente00: Llegó
 0.0000 Cliente00: Esperó  0.000
 2.5256 Cliente00: Acabó
 4.5903 Cliente01: Llegó
 4.5903 Cliente01: Esperó  0.000
 5.7269 Cliente02: Llegó
 8.5431 Cliente01: Acabó
 8.5431 Cliente02: Esperó  2.816
11.0077 Cliente02: Acabó
15.7497 Cliente03: Llegó
15.7497 Cliente03: Esperó  0.000
15.8858 Cliente04: Llegó
16.0069 Cliente05: Llegó
17.0045 Cliente06: Llegó
18.3642 Cliente03: Acabó
18.3642 Cliente04: Esperó  2.478
21.6236 Cliente04: Acabó
21.6236 Cliente05: Esperó  5.617
21.7272 Cliente07: Llegó
21.7565 Cliente08: Llegó
25.3057 Cliente05: Acabó
25.3057 Cliente06: Esperó  8.301
28.8148 Cliente06: Acabó
28.8148 Cliente07: Esperó  7.088
29.1319 Cliente09: Llegó
31.8294 Cliente07: Acabó
31.8294 Cliente08: Esperó 10.073
31.8294 Cliente08: Abandonó después de esperar 10.073
31.8294 Cliente09: Esperó  2.698
35.9321 Cliente09: Acabó


In [ ]:
#@title Modificar el código para calcular número de abandonos, tiempo promedio en el sistema, tiempo promedio en cola contando únicamente los que esperaron y contando todos los clientes.

import simpy
import random

RANDOM_SEED = 42
NEW_CUSTOMERS = 10  # número total de clientes
INTERVAL_CUSTOMERS = 4.5  # Generar nuevos clientes cada x minutos
MAX_WAIT_TIME = 10.0  # Máximo tiempo de espera en minutos

# Variables para rastrear estadísticas
abandonos = 0
tiempo_promedio_en_sistema = 0
tiempo_promedio_en_cola_esperando = 0
tiempo_promedio_en_cola_total = 0

# Generación de llegadas distribuidas exponencialmente
def llegadas(env, num, tasa, caja):
    global abandonos, tiempo_promedio_en_sistema, tiempo_promedio_en_cola_esperando, tiempo_promedio_en_cola_total
    for i in range(num):
        c = cliente(env, 'Cliente%02d' % i, caja, mus=3.2, stds=0.6)
        env.process(c)
        t = random.expovariate(1.0 / tasa)
        yield env.timeout(t)

# Cada cliente requiere la caja y tiene un tiempo de servicio aprox. N(mu, stds)
def cliente(env, name, caja, mus, stds):
    global abandonos, tiempo_promedio_en_sistema, tiempo_promedio_en_cola_esperando, tiempo_promedio_en_cola_total
    # Llega un cliente, registrar momento
    arrive = env.now
    print('%7.4f %s: Llegó' % (arrive, name))

    with caja.request() as req:
        yield req
        wait = env.now - arrive
        tiempo_promedio_en_cola_total += wait  # Incluye tiempo en cola para todos los clientes
        if wait > 0:  # Verifica si el cliente esperó en cola
            tiempo_promedio_en_cola_esperando += wait  # Incluye tiempo en cola solo para los que esperaron
        print('%7.4f %s: Esperó %6.3f' % (env.now, name, wait))

        # Comprueba si el cliente ha esperado más de 10 minutos, y si es así, lo hace abandonar.
        if wait > MAX_WAIT_TIME:
            print('%7.4f %s: Abandonó después de esperar %6.3f' % (env.now, name, wait))
            abandonos += 1
            return

        tib = random.normalvariate(mus, stds)
        yield env.timeout(tib)
        print('%7.4f %s: Acabó' % (env.now, name))
        tiempo_en_sistema = env.now - arrive
        tiempo_promedio_en_sistema += tiempo_en_sistema


print('Caja\n')
random.seed(RANDOM_SEED)
env = simpy.Environment()

caja = simpy.Resource(env, capacity=1)
env.process(llegadas(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, caja))
env.run()

tiempo_promedio_en_sistema /= NEW_CUSTOMERS
tiempo_promedio_en_cola_esperando /= NEW_CUSTOMERS
tiempo_promedio_en_cola_total /= NEW_CUSTOMERS

print(f'Número de abandonos: {abandonos}')
print(f'Tiempo promedio en el sistema: {tiempo_promedio_en_sistema:.3f} minutos')
print(f'Tiempo promedio en cola para los que esperaron: {tiempo_promedio_en_cola_esperando:.3f} minutos')
print(f'Tiempo promedio en cola para todos los clientes: {tiempo_promedio_en_cola_total:.3f} minutos')


Caja

 0.0000 Cliente00: Llegó
 0.0000 Cliente00: Esperó  0.000
 2.5256 Cliente00: Acabó
 4.5903 Cliente01: Llegó
 4.5903 Cliente01: Esperó  0.000
 5.7269 Cliente02: Llegó
 8.5431 Cliente01: Acabó
 8.5431 Cliente02: Esperó  2.816
11.0077 Cliente02: Acabó
15.7497 Cliente03: Llegó
15.7497 Cliente03: Esperó  0.000
15.8858 Cliente04: Llegó
16.0069 Cliente05: Llegó
17.0045 Cliente06: Llegó
18.3642 Cliente03: Acabó
18.3642 Cliente04: Esperó  2.478
21.6236 Cliente04: Acabó
21.6236 Cliente05: Esperó  5.617
21.7272 Cliente07: Llegó
21.7565 Cliente08: Llegó
25.3057 Cliente05: Acabó
25.3057 Cliente06: Esperó  8.301
28.8148 Cliente06: Acabó
28.8148 Cliente07: Esperó  7.088
29.1319 Cliente09: Llegó
31.8294 Cliente07: Acabó
31.8294 Cliente08: Esperó 10.073
31.8294 Cliente08: Abandonó después de esperar 10.073
31.8294 Cliente09: Esperó  2.698
35.9321 Cliente09: Acabó
Número de abandonos: 1
Tiempo promedio en el sistema: 5.812 minutos
Tiempo promedio en cola para los que esperaron: 3.907 minutos
Tiemp

#REPRODUCCIÓN DEL MODELO EN GPSS

In [ ]:
GENERATE 10, 4.5, 0
QUEUE cola
SERVER caja 1

DELAY tiempo_llegada
    ENTER cola
    DEPART cola

        IF tiempo_en_cola > 10 THEN
            TERMINATE

        ELSE
            DELAY tiempo_servicio
            DEPART caja
        END

TERMINATE
